# Milestone 2 - Cleaning a Flat File - Rachel Wantuch

What changes we made to the data?
The first change made to the data was making the second row the header. The initial data had the first 6 header slots as Nan and then 82 occurances of 'Missed Games'. None of that was helpful so I basically told it to ignore that column.

The second change was adding two columns to the dataframe. I did this for future data manipulations to have some place to live.

The third transformation using rows labeled "1" to "82" to fill in the new "Games Missed' Column.

The 4th transformation was eliminating rows where NaN showed up in the Cap Hit and/or Chip columns. This was just a personal preference.

The 5th transformation is still in progress. I want to convert the Cap Hit and Chip column to read as numbers. Currently they are sitting as strings with $ signs and m for million and k for thousands. I need to finish writing a function to conver those rows.

The 6th transformation was seperating the players first and last names and getting rid of the old player column after that.

Are there any legal or regulatory guidelines for your data or project topic?
There are no guidelines that I am currently aware of.

What risks could be created based on the transformations done?
By leaving the first name as its own entity I can see the possibility for someone filtering based on say the name 'John' and getting data for 8 different players and not knowing it.

Did you make any assumptions in cleaning/transforming the data?
I will be making an assumption in cleaning the Chip data once I write the function. For instnace there are a few rows where the author has 1,256K. I'm assuming it doesn't mean to add three more 0's and its actualy just saying 1,256. I think anything below $10000 was written that way.

How was your data sourced/verified for credibility?
The author of the original data provided links to where he pulled that data from.

Was your data acquired in an ethical way?
Yes. I used Tableau's download feature to obtain the flat file.

How would you mitigate any of the ethical implications you have identified?
I don't see any ethical implications in this data, but a good way to validate it is to provide sourcfes whenever possible. Another option is to also provide metadata for the users to know what each column means and how it was figured.

In [310]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:/Users/rwant/OneDrive/Desktop/Bellevue/DSC 540/Final/NHL Injury Database.csv", skiprows=[0])
df.head()

,Season,Team,Position,Player,Injury Type,Cap Hit,Chip,1,2,3,...,73,74,75,76,77,78,79,80,81,82
0,2000/01,Anaheim,F,"Kariya, Paul",Foot,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000/01,Anaheim,F,"Leclerc, Mike",Abdominal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000/01,Anaheim,F,"Leclerc, Mike",Knee,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000/01,Anaheim,F,"McDonald, Andy",Concussion,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000/01,Anaheim,F,"McInnis, Marty",Groin,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering it down to season's 2023/24

In [313]:
#Finding the first occurance of the 2014/15 by index location
values = "2023/24"
row_index=df[df['Season']==values].index
print(row_index)

Index([20984, 20985, 20986, 20987, 20988, 20989, 20990, 20991, 20992, 20993,
       ...
       21869, 21870, 21871, 21872, 21873, 21874, 21875, 21876, 21877, 21878],
      dtype='int64', length=895)


In [315]:
df=df.iloc[20984:21878].reset_index(drop=True)

In [317]:
df.columns

Index(['Season', 'Team', 'Position', 'Player', 'Injury Type', 'Cap Hit',
       'Chip', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72',
       '73', '74', '75', '76', '77', '78', '79', '80', '81', '82'],
      dtype='object')

## Transformation 2: Inserting New Columns (Games Missed)

It appears that the original author of this dataset had a unique set up for how many players missed games. If they missed 6 games he put the number 6 in the Games Missed 6 column. This seems redundant so I want to merge all those columns into one singular column.

In [321]:
df['Games Missed']= np.nan

## Tranformation 3: Summing Columns labeled "1" to "82" and adding that to Games Missed column

In [324]:
df['Games Missed']=df.iloc[:,7:89].apply(lambda row:'-'.join(row.dropna().astype(str)),axis=1)

In [325]:
df1=df.drop(df.iloc[:,7:89],axis=1)

## Transformation 4: Eliminating rows where NaN appears in Cap Hit and Chip.

I am going to eliminate all rows without salary data for players. For the purpose of this project that should be ok. I just want to deal with rows with all the data.

In [330]:
df=df1.dropna(subset=['Cap Hit', 'Chip'])

## Transformation 4.1 Change Player Name to First Name Last Name to match other data sources I used.

In [333]:
column=df1.pop("Player")
df1.insert(1,"Player",column)

In [335]:
df1[['Last Name', 'First Name']]=df1['Player'].str.split(',',expand=True)

In [337]:
df1=df1.drop('Player',axis=1)

In [339]:
column=df1.pop("First Name")
df1.insert(1,"First Name",column)
column=df1.pop("Last Name")
df1.insert(2,"Last Name",column)

df1['PlayerName']=df1['First Name'] + ' ' + df1['Last Name']

In [341]:
df1=df1.drop('First Name', axis=1)
df1=df1.drop('Last Name', axis=1)
column=df1.pop("PlayerName")
df1.insert(1,"PlayerName",column)

## Transformation 5: Remove Season COlumn

In [344]:
df1=df1.drop('Season', axis=1)

## Convert Chip column to dollars

In [347]:
def convert_chip(x):
    x1=x.replace('k','000')
    return x1

In [349]:
df1['Chip_con']=df1['Chip'].apply(convert_chip)
df1.head()

,PlayerName,Team,Position,Injury Type,Cap Hit,Chip,Games Missed,Chip_con
0,Leo Carlsson,Anaheim,F,Concussion,$4.200m,$410k,8.0,$410000
1,Leo Carlsson,Anaheim,F,Knee,$4.200m,$512k,10.0,$512000
2,Leo Carlsson,Anaheim,F,Lower body,$4.200m,$102k,2.0,$102000
3,Adam Henrique,Anaheim,F,Illness,$5.825m,$71k,1.0,$71000
4,Ross Johnston,Anaheim,F,Illness,$1.100m,$13k,1.0,$13000


In [351]:
df1=df1.drop('Chip', axis=1)
column=df1.pop("Chip_con")
df1.insert(5,"Chip_con",column)
pd.set_option('display.max_rows',None)

In [353]:
df1['Chip_con']=df1['Chip_con'].str.replace(',','')
df1['Chip_con']=df1['Chip_con'].str.replace('$','')
df1['Chip_con']=df1['Chip_con'].astype(int)

## Condense Columns so each row is for one player with all injuries listed in one segment an

In [356]:
df1['Games Missed']=df1['Games Missed'].astype(float)
df1.head()

,PlayerName,Team,Position,Injury Type,Cap Hit,Chip_con,Games Missed
0,Leo Carlsson,Anaheim,F,Concussion,$4.200m,410000,8.0
1,Leo Carlsson,Anaheim,F,Knee,$4.200m,512000,10.0
2,Leo Carlsson,Anaheim,F,Lower body,$4.200m,102000,2.0
3,Adam Henrique,Anaheim,F,Illness,$5.825m,71000,1.0
4,Ross Johnston,Anaheim,F,Illness,$1.100m,13000,1.0


In [358]:
import locale
locale.setlocale(locale.LC_ALL,'en_US.UTF-8')
def currency(x):
    return locale.currency(x,grouping=True)

In [360]:
df1=df1.groupby('PlayerName', as_index=False).agg({'Chip_con':'sum','Games Missed':'sum'})
df1.head()

,PlayerName,Chip_con,Games Missed
0,A.J. Greer,167000,18.0
1,Aaron Ekblad,2743000,31.0
2,Adam Boqvist,1014000,32.0
3,Adam Fantilli,1670000,33.0
4,Adam Fox,1159000,10.0


In [362]:
df1['Chip']=df1['Chip_con'].apply(currency)
df1=df1.drop('Chip_con', axis=1)

df1=df1.rename(columns={'Games Missed':'Games_Missed'})
df1['PlayerName']=df1['PlayerName'].str.strip()

In [364]:
print(df1)

                   PlayerName  Games_Missed            Chip
0                  A.J. Greer          18.0     $167,000.00
1                Aaron Ekblad          31.0   $2,743,000.00
2                Adam Boqvist          32.0   $1,014,000.00
3               Adam Fantilli          33.0   $1,670,000.00
4                    Adam Fox          10.0   $1,159,000.00
5               Adam Henrique           1.0      $71,000.00
6                 Adam Pelech          24.0   $1,682,000.00
7                  Adam Raška           3.0      $33,000.00
8                Adam Ružička           4.0      $37,000.00
9                   Adin Hill          29.0   $1,314,000.00
10               Adrian Kempe           5.0     $335,000.00
11              Alec Martinez          21.0   $1,344,000.00
12          Aleksander Barkov           9.0   $1,098,000.00
13             Alex Goligoski          15.0     $366,000.00
14               Alex Killorn          19.0   $1,448,000.00
15                  Alex Lyon           

In [366]:
df1.to_csv("player_injuries.csv", index=False)